In [2]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import Button, Dropdown, HBox, HTML, Label, Layout, Output, VBox
from ipycanvas import Canvas
from IPython.display import display, clear_output

rng = np.random.default_rng()
power_limit = np.arange(0, 100, 1) / 100
min_in_day = 24*60
minute_power = None
custom = False

def setup_canvas():
    global canvas
    canvas.clear()
    canvas.fill_style = "rgba(0, 0, 0, 0)"  # Fully transparent background
    canvas.fill_rect(0, 0, canvas_width, canvas_height)
    # Draw grid and axes
    canvas.line_width = 1
    canvas.stroke_style = "rgba(1, 1, 1, 0.5)"
    canvas.stroke_line(0, canvas_height, canvas_width, canvas_height)
    canvas.stroke_line(0, canvas_height, 0, 0)
    canvas.stroke_style = "rgba(15, 15, 15, 0.5)"
    for i in range(1, 12):
        y = canvas_height / 12 * i
        canvas.stroke_line(0, y, canvas_width, y)
    for i in range(1, 12):
        x = canvas_width / 12 * i
        canvas.stroke_line(x, 0, x, canvas_height)
    canvas.stroke_style = "rgba(0, 0, 0, 1)"
    canvas.line_width = 4

def on_capture_button_click(b):
    global minute_power, custom
    img_data = canvas.get_image_data()
    minute_power = np.zeros(min_in_day)
    for i in range(min_in_day):
        cx = int(i / min_in_day * canvas_width)
        for cy in range(canvas_height):
            if img_data[cy, cx, 3] > 200:
                minute_power[i] = (canvas_height - cy) / canvas_height
                break
    minute_power = np.tile(minute_power, 365)
    custom = True
    update_distribution('Custom')

def update_distribution(distribution):
    global minute_power, custom, graphs
    if distribution == 'Custom' and not custom:
        canvasbox.layout.display = 'block'
        setup_canvas()
        return
    elif distribution != 'Custom':
        if custom:
            canvasbox.layout.display = 'none'
            custom = False
        match distribution:
            case 'Uniform':
                minute_power = rng.uniform(0, 1, 365*24*60)
            case 'Gaussian':
                minute_power = rng.normal(0.5, 0.1, 365*24*60)
                minute_power = np.clip(minute_power, 0, 1)

    tot = max(np.sum(minute_power), 1)
    hist, bin_edges = np.histogram(minute_power, bins=100, density=True)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    hist = hist / np.sum(hist)
    curtailed = [np.sum(np.maximum(0, minute_power - t)) / tot for t in power_limit]
    graphs.clear_output()
    with graphs:
        fig = plt.figure(figsize=(15, 5))
        ax1 = plt.subplot(131)
        ax1.plot(bin_centers, hist)
        ax1.set_xlabel('Power level')
        ax1.set_ylabel('Relative frequency (PDF)')
        ax1.set_ylim(0)
        ax1.set_xlim(0)
        ax1.set_title('Distribution')
        
        ax2 = plt.subplot(132)
        ax2.plot(minute_power[:1440])
        ax2.set_xlabel('Minutes')
        ax2.set_ylabel('Power level')
        ax2.set_ylim(0, 1)
        ax2.set_title('24 Hours of power data')
        
        ax3 = plt.subplot(133)
        ax3.plot(power_limit, curtailed)
        ax3.set_xlabel('Power Limit')
        ax3.set_ylabel('Curtailed Energy Fraction')
        ax3.set_ylim(0, 1)
        ax3.set_title('Curtailment')
        plt.tight_layout()
        plt.show()

# Canvas setup
canvas_width, canvas_height = 400, 200
canvas = Canvas(
    width=canvas_width,
    height=canvas_height,
    sync_image_data=True,
    layout={'border': '1px solid black', 'width': f'{canvas_width}px', 'margin-left': '50px'}
)
is_drawing, last_x, last_y = False, 0, 0
y_axis = HTML(f'''
    <div style="position:absolute; left:0; top:0; width:50px; height:{canvas_height}px;">
        <div style="position:absolute; top:-5px; right:5px;">1.0</div>
        <div style="position:absolute; top:{canvas_height-25}px; right:5px;">0.0</div>
    </div>
''')
x_axis = HTML(f'''
    <div style="margin-left:50px; width:{canvas_width}px; height:30px; position:relative;">
        <div style="position:absolute; left:0; top:0;">0:00</div>
        <div style="position:absolute; left:{canvas_width/2-15}px; top:0;">12:00</div>
        <div style="position:absolute; left:{canvas_width-15}px; top:0;">23:59</div>
    </div>
''')
cbox = HBox([canvas], layout=Layout(margin='0 0 0 50px'))
canvas_and_labels = VBox([
    HBox([y_axis, cbox], layout=Layout(height=f'{canvas_height}px')),
    x_axis
])
btn_value = Button(description="Use these values")
btn_clear = Button(description="Clear")
btn_value.on_click(on_capture_button_click)
btn_clear.on_click(lambda _: setup_canvas())
canvasbox = VBox([ Label(value="Draw power generation over 24h"),
                   canvas_and_labels,
                   HBox([btn_value, btn_clear])
                 ])
canvasbox.layout.display = 'none'

@canvas.on_mouse_down
def mouse_down(x, y):
    global is_drawing, last_x, last_y
    is_drawing, last_x, last_y = True, x, y
@canvas.on_mouse_move
def mouse_move(x, y):
    global is_drawing, last_x, last_y
    if is_drawing:
        canvas.stroke_line(last_x, last_y, x, y)
        last_x, last_y = x, y
@canvas.on_mouse_up
def mouse_up(x, y):
    global is_drawing
    is_drawing = False
@canvas.on_mouse_out
def mouse_out(x, y):
    global is_drawing
    is_drawing = False

dropdown_default = 'Uniform'
sel_distribution = Dropdown(options=['Uniform', 'Gaussian', 'Custom'], value=dropdown_default, description='Distribution:')
sel_distribution.observe(lambda change: update_distribution(change.new), names='value')
graphs = Output()
update_distribution(dropdown_default)
display(VBox([sel_distribution, graphs, canvasbox]))
